In [2]:
import rasterio
import numpy as np
import plotly.graph_objects as go

def read_raster(file_path):
    with rasterio.open(file_path) as src:
        raster = src.read(1)
        profile = src.profile
    return raster, profile

In [18]:
def count_pixels(raster):
    nan_pixels = np.sum(np.isnan(raster))
    low_range_pixels = np.sum(raster < -5)
    mid_range_pixels = np.sum((raster >= -5) & (raster <= 5))
    high_range_pixels = np.sum(raster > 5)

    total_pixels = nan_pixels + low_range_pixels + mid_range_pixels + high_range_pixels
    return nan_pixels, low_range_pixels, mid_range_pixels, high_range_pixels, total_pixels

def plot_pie_chart(nan_pixels, low_range_pixels, mid_range_pixels, high_range_pixels, total_pixels, output_file):
    categories = ['Nan Pixels', 'Loss', 'Neutral', 'Gain']
    values = [nan_pixels, low_range_pixels, mid_range_pixels, high_range_pixels]

    percentages = [value / total_pixels * 100 for value in values]

    colors = ['fuchsia', 'red', 'gray', 'royalblue']  # Custom colors for each range

    fig = go.Figure(go.Pie(
        labels=categories,
        values=percentages,
        hole=0.3,
        marker_colors=colors  # Apply custom colors to pie chart slices
    ))

    # Remove the original title from the Pie object
    fig.update_traces(title=None)

    # Create a separate layout title with desired formatting
    fig.update_layout(
        title_text="Pixel Distribution in Trend",  # Single title with desired text
        title_x=0.5,  # Center the title horizontally
        title_font_size=20,  # Increase title font size
        title_font_weight="bold"  # Make title bold for emphasis
    )

    fig.write_image(output_file, scale=2)
    fig.show()

raster_file = 'C:/Users/Facu/Desktop/Malawi/trend.tif'
output_file = 'C:/Users/Facu/Desktop/Malawi/biomass_pixel_pie_chart.png'

raster, profile = read_raster(raster_file)
nan_pixels, low_range_pixels, mid_range_pixels, high_range_pixels, total_pixels = count_pixels(raster)
plot_pie_chart(nan_pixels, low_range_pixels, mid_range_pixels, high_range_pixels, total_pixels, output_file)

In [19]:
def read_raster(file_path):
    with rasterio.open(file_path) as src:
        raster = src.read(1)
        profile = src.profile
    return raster, profile

def plot_bar_chart(raster, output_file):
    negative_ranges = ['<-30', '-27 to -24', '-21 to -18', '-15 to -12', '-9 to -6']  # Reduced ranges (5 parts)
    negative_values = raster[raster < 0].flatten()
    negative_values = negative_values[~np.isnan(negative_values)]

    bins = [-np.inf, -30, -27, -24, -21, -18, -15, -12, -9, -6, 0]  # Adjusted bins based on ranges
    hist, _ = np.histogram(negative_values, bins=bins)

    # Scale pixel count to hectares (same as before)
    hist = hist * 0.3

    fig = go.Figure(go.Bar(
        x=negative_ranges,
        y=hist,
        name="Rangos Negativos",
        marker=dict(color='rgba(255, 0, 0, 1.0)')  # Red color for negative ranges
    ))

    fig.update_layout(
        title_text="Distribution of Negative Ranges in Hectares",
        xaxis_title="Loss measured in Tn/Ha",  # Label for x-axis (Range of Values)
        yaxis_title="Surface in Ha",# Single title with desired text
        title_x=0.5,  # Center the title horizontally
        title_font_size=20,  # Increase title font size
        title_font_weight="bold"  # Make title bold for emphasis
    )

    fig.write_image(output_file, scale=2)
    fig.show()

raster_file = 'C:/Users/Facu/Desktop/Malawi/trend.tif'
output_file = 'C:/Users/Facu/Desktop/Malawi/biomass_negative_ranges_bar_chart.png'

raster, profile = read_raster(raster_file)
plot_bar_chart(raster, output_file)

In [15]:
def calculate_average(raster_files):
    avg_values = []
    for file in raster_files:
        raster, _ = read_raster(file)
        avg_values.append(np.nanmean(raster))
    return avg_values

def plot_line_chart(avg_values, output_file):
    years = ['2019', '2020', '2021']

    fig = go.Figure(go.Scatter(
        x=years,
        y=avg_values,
        mode='lines+markers',
        name='Promedio Anual'
    ))

    fig.update_layout(
        title_text="Time series",
        yaxis_title="Average in Tn/Ha",# Single title with desired text
        title_x=0.5,  # Center the title horizontally
        title_font_size=20,  # Increase title font size
        title_font_weight="bold"  # Make title bold for emphasis
    )

    # Guardar la imagen con una resolución mayor
    fig.write_image(output_file, scale=2)

    fig.show()

raster_files = [
    'C:/Users/Facu/Desktop/Malawi/Malawi_2019_complete.tif',
    'C:/Users/Facu/Desktop/Malawi/Malawi_2020_complete.tif',
    'C:/Users/Facu/Desktop/Malawi/Malawi_2021_complete.tif',
]
output_file = 'C:/Users/Facu/Desktop/Malawi/annual_pixel_average.png'

avg_values = calculate_average(raster_files)
plot_line_chart(avg_values, output_file)